In [0]:
from IPython.display import display, Javascript, HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import uuid
import os


class GoogleDriveDatabase:
  def __init__(self, drive, DATABASE_GID:str):
    assert isinstance(DATABASE_GID, str)
    self.folders = {}
    self.drive = drive
    folder_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(DATABASE_GID)}).GetList()
    for file in folder_list:
      if file['mimeType'] == "application/vnd.google-apps.folder":
        self.folders[file['title']] = file['id']
    print("{} folders loaded".format(len(self.folders.keys())))
  def upload(self, filename, character, fileType):
    assert isinstance(filename, str)
    assert isinstance(character, str)
    assert isinstance(fileType, str)
    FILETYPE_MIME_MAP = {
        "jpeg" : "image/jpeg",
        "json" : "application/json",
        "zip" : "application/zip"
    }
    assert fileType in FILETYPE_MIME_MAP.keys(), "fileType must be one of the following: {}".format(fileType)
    assert os.path.isfile(filename), "{} does not exist as a file".format(filename)
    assert self.checkFolder(character), "{} is not a valid character. Pick from list: \n{}".format(character, tuple(self.folders.keys()))
    file = self.drive.CreateFile({
        "title" :  os.path.split(filename)[1],
        "mimeType" : FILETYPE_MIME_MAP[fileType],
        "parents" : [{"id" : self.folders[character]}]
    })
    file.SetContentFile(filename)
    file.Upload()
    os.remove(filename)
    print("uploaded and deleted {}".format(filename))

  @staticmethod
  def FILE_EXTENSIONS() -> list:
    return [".jpg", ".jpeg", ".png", ".zip", ".json"]

  def getFiles_filename(self, filename, character=None) -> list:
    if character:
      return [x for x in self.getFiles(character) if x['title'] == filename]
    q="name = 'file_name'"
    return self.drive.ListFile({"q" : "title = '{}'".format(filename)}).GetList()

  def getFiles(self, character) -> list:
    return [
      x for x in self.drive.ListFile({'q': "'{}' in parents and trashed=false".format(self.folders[character])}).GetList()
      if x['mimeType'] != "application/vnd.google-apps.folder" 
    ]
  def download_file_name(self, file_name):
      if not any(extension in file_name for extension in GoogleDriveDatabase.FILE_EXTENSIONS()):
        file_name += ".jpg"
      return file_name
  def download_file(self, file, folder:str, **kwargs) -> str:
      check_already_exist = kwargs.get("check_local", False)
      file_name = os.path.join(folder, file['title'])
      file_name = self.download_file_name(file_name)
      if check_already_exist:
        local_files = [os.path.join(folder, file) for file in os.listdir(folder)]
        if file_name in local_files:
          return file_name
      file.GetContentFile(file_name)
      print("downloaded", file_name)
      return file_name

  def download(self, character:str,folder:str):
    file_list = self.getFiles(character)
    os.makedirs(folder, exist_ok=True)
    returnlist = []
    
    for file in file_list:
      returnlist.append(self.download_file(file, folder, check_local=True))


    return tuple(returnlist)

  def checkFolder(self, name):
    assert isinstance(name, str)
    return name in self.folders.keys()

auth.authenticate_user() # Google auth stuff, make sure to sign in with your ucsb account
gauth = GoogleAuth() # Google auth stuff
gauth.credentials = GoogleCredentials.get_application_default() # Google auth stuff
drive = GoogleDrive(gauth) # Google auth stuff

CHARACTER_DATABASE = "1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC"
character_database = GoogleDriveDatabase(drive, CHARACTER_DATABASE)

def showImage(url):
  # js = Javascript('''
  #  function showImage(url){
  #    return 4
  #  }
  # ''')
  # dispaly(js)
  # eval_js('showImage({})'.format(url))
  display(HTML('<img src="{}">'.format(url)))

while True:
  file_name = input("What is the name of the picture to remove from the dataset?\n") # A_0f3d575f-dc97-4b50-b7bd-4ba895447ecb.jpg
  found_files = character_database.getFiles_filename(file_name)
  if len(found_files) == 0:
    print("No files were found")
    continue
  print("{} file found".format(len(found_files)))
  i = 0
  for x in found_files:
    file_downloaded = character_database.download_file(x, "/content/")
  print(found_files[0])
  # showImage(found_files[0]["embedLink"])
  # showImage("file:/" + file_downloaded)
  display(Image(file_downloaded))
  if input("Do you want to delete this from the dataset? [y/N]\n") == "y":
    [x.Delete() for x in found_files]
  
